In [1]:
import os
import soundfile
import re
import numpy as np
import librosa
import pandas as pd
from progressbar import progressbar
from datasets import load_metric
from pydub import AudioSegment
import random
random.seed(10)

In [2]:
from datasets import load_dataset, load_metric, Audio
common_voice_test = load_dataset("common_voice", "ta", split="test")

Reusing dataset common_voice (/home/ubuntu/.cache/huggingface/datasets/common_voice/ta/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [3]:
common_voice_test

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 1781
})

In [4]:
data_frame = pd.DataFrame()
data_frame["audios"] = list(common_voice_test["path"])
data_frame["text"] = list(common_voice_test["sentence"])

In [5]:
variation = "Native"
e_type="base"
noise_variation = "laughter" # rain, wind, laughter

In [6]:
# removing special characters from the transcripts in the column data_frame["text"]

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(sentence):
    sentence = re.sub(chars_to_ignore_regex, '', sentence)
    return sentence

data_frame["text"] = data_frame["text"].apply(lambda x: remove_special_characters(x))

In [7]:
data_frame.head(3)

,audios,text
0,/home/ubuntu/.cache/huggingface/datasets/downl...,தாயும் பிள்ளையும் சரேலென் றெழுந்தே
1,/home/ubuntu/.cache/huggingface/datasets/downl...,புனிதரும் நீயும் என்புந்தி எந்நாளும் பொருந்துகவே
2,/home/ubuntu/.cache/huggingface/datasets/downl...,கதை உத்திகள் அவரிடம் படிந்துபோன விஷயம்


In [8]:
# viewing path of a single file
print(data_frame["audios"][1])

/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/bbc7e5ef7e1baed3b86a20ce9e146e069a26e1bebd1feadc09a550c84ff03cf0/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_23721822.mp3


In [9]:
# check one file from the data_frame for specifications
audio_array, sampling_rate = librosa.load(data_frame["audios"][1])

print(sampling_rate)

/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


22050


In [10]:
# resampling audio file to 16 KHz

audio_array, sampling_rate = librosa.load(data_frame["audios"][1], sr=16000)
print(sampling_rate)

/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


16000


In [11]:
# getting unique words in the transcripts to use it with the language modeler

words = " ".join(list(data_frame["text"])).split()
unique_words = list(set(words))
print(len(unique_words))

5797


### Prediction

In [12]:

import config
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

transcriber_processor = Wav2Vec2Processor.from_pretrained("wav2vec2-large-rbg-tamil-native/")
transcriber_processor.tokenizer.do_lower_case = True
transcriber_encoder_model = Wav2Vec2ForCTC.from_pretrained("wav2vec2-large-rbg-tamil-native/")

transcriber_encoder_model  = transcriber_encoder_model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# noise folder path
noise_folder = "noise/matrix/"

rain_noises = ['1-56311-A-10.wav', '1-63871-A-10.wav', '2-101676-A-10.wav', '2-117625-A-10.wav', '5-181766-A-10.wav', '5-188655-A-10.wav', '3-157487-A-10.wav', '3-157615-A-10.wav', '4-160999-A-10.wav', '4-161127-A-10.wav']
wind_noises = ['1-51037-A-16.wav', '1-69760-A-16.wav', '2-104952-A-16.wav', '2-104952-B-16.wav', '3-246513-A-16.wav', '3-246513-B-16.wav', '4-144083-A-16.wav', '4-144083-B-16.wav', '5-157204-A-16.wav', '5-157204-B-16.wav']
laughter_noises = ['1-72695-A-26.wav', '1-73123-A-26.wav', '2-109759-A-26.wav','2-109759-B-26.wav', '3-152912-A-26.wav', '3-152997-A-26.wav', '4-132803-A-26.wav', '4-132810-A-26.wav', '5-242932-B-26.wav','5-244526-A-26.wav']

rain_noises = [os.path.join(noise_folder, file) for file in rain_noises]
wind_noises = [os.path.join(noise_folder, file) for file in wind_noises]
laughter_noises = [os.path.join(noise_folder, file) for file in laughter_noises]

noise_dict = {'rain': rain_noises, 'wind': wind_noises, 'laughter': laughter_noises}

In [14]:
# prediction against base model with and without language modeler

def get_transcriptions(audio_path, unique_words, e_type="base", noise_type="none"):
    
    if noise_type == "none":
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)
    else:
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)
        
        soundfile.write("tem.wav",audio_array, samplerate=16000)
        
        GAIN_CONSTANT = -34.
        NOISE_VOLUME=0.3
        
        noise_file = random.choice(noise_dict[noise_type])
        audio = AudioSegment.from_file("tem.wav", 'wav')
        audio = audio.normalize()
        background = AudioSegment.from_file(noise_file, 'wav')
        background = background.normalize()
        background = background.apply_gain(GAIN_CONSTANT * (1. - 0.3))
        output = audio.overlay(background, position=0, loop=True)
        output = output.set_frame_rate(16000)
        output.export("temp.wav", format='wav')
        audio_array, sampling_rate = librosa.load("temp.wav", sr=16000)
        
    inputs = transcriber_processor(audio_array, sampling_rate=16000,
                                       return_tensors="pt", padding=True)
    inputs = inputs.to("cuda")
    with torch.no_grad():
            character_probability = transcriber_encoder_model(inputs.input_values,
                                                              attention_mask=inputs.attention_mask).logits
    inputs = inputs.to("cpu")
    predicted_ids = torch.argmax(character_probability, dim=-1)
    encoder_text = transcriber_processor.batch_decode(predicted_ids)[0]
    
    decoder_text = "None" # "not disclosed"
    
    return encoder_text, decoder_text

In [15]:
without_lm_op = list()
with_lm_op = list()

for item in progressbar(data_frame["audios"]):
    e_text, d_text = get_transcriptions(item, unique_words, e_type="base_model", noise_type=noise_variation)
    without_lm_op.append(e_text)
    with_lm_op.append(d_text)

data_frame["without_lm"] = without_lm_op
data_frame["with_lm"] = with_lm_op
data_frame.to_csv("results/CommonVoice-Tamil-"+ e_type  + "-" + variation + "-" + noise_variation + ".tsv", sep="\t", index=False)

  0% (0 of 1781) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a

  0% (10 of 1781) |                      | Elapsed Time: 0:00:12 ETA:   0:32:10/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
  0% (11 of 1781) |                      | Elapsed Time: 0:00:13 ETA:   0:34:16/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Tryin

/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
  1% (21 of 1781) |                      | Elapsed Time: 0:00:24 ETA:   0:29:37/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds t

  1% (31 of 1781) |                      | Elapsed Time: 0:00:40 ETA:   0:37:50/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
  1% (32 of 1781) |                      | Elapsed Time: 0:00:41 ETA:   0:33:05/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Tryin

/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
  2% (42 of 1781) |                      | Elapsed Time: 0:00:57 ETA:   0:49:55/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds t

...

 97% (1732 of 1781) |################### | Elapsed Time: 0:32:30 ETA:   0:01:13/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
 97% (1733 of 1781) |################### | Elapsed Time: 0:32:33 ETA:   0:01:46/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Tryin

/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
 97% (1743 of 1781) |################### | Elapsed Time: 0:32:48 ETA:   0:01:24/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds t

 98% (1753 of 1781) |################### | Elapsed Time: 0:33:04 ETA:   0:00:47/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
 98% (1754 of 1781) |################### | Elapsed Time: 0:33:05 ETA:   0:00:48/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Tryin

/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
 99% (1764 of 1781) |################### | Elapsed Time: 0:33:20 ETA:   0:00:21/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds t

 99% (1774 of 1781) |################### | Elapsed Time: 0:33:36 ETA:   0:00:11/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/ubuntu/AISS/common/transformers/models/wav2vec2/modeling_wav2vec2.py:875: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
 99% (1775 of 1781) |################### | Elapsed Time: 0:33:37 ETA:   0:00:09/home/ubuntu/environments/aiss/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Tryin

In [16]:
data_frame.head(5)

,audios,text,without_lm,with_lm
0,/home/ubuntu/.cache/huggingface/datasets/downl...,தாயும் பிள்ளையும் சரேலென் றெழுந்தே,தாயும் பிள்ளையும் சரலின் நிடுருந்தே,தாயும் பிள்ளையும் சரேலென் றெழுந்தே
1,/home/ubuntu/.cache/huggingface/datasets/downl...,புனிதரும் நீயும் என்புந்தி எந்நாளும் பொருந்துகவே,உரிததும மீயும் என்கும்பி என்னாலும் கழுிபுகதே,மரத்து நீயும் என் கும்மி என்னாகும் பொருந்துகவே
2,/home/ubuntu/.cache/huggingface/datasets/downl...,கதை உத்திகள் அவரிடம் படிந்துபோன விஷயம்,கதைமுத்திக லாவரிபம் படிந்து கோணஞ்சேன்,கதை முத்த அவரிடம் படிந்து கோலம் சேர்
3,/home/ubuntu/.cache/huggingface/datasets/downl...,நீங்கள் நேற்று கீரைப் பொறியல் சாப்பிட்டீர்களா,நீங்கள் நேற்று கீரை பொருயல் சாப்பிட்டீர்களா,நீங்கள் நேற்று கீரைப் பொறியல் சாப்பிட்டீர்களா
4,/home/ubuntu/.cache/huggingface/datasets/downl...,வைத்துள தமிழர் நூற்கள்,வைத்தொள்ளத் தமிழர் ஞூர்கள்,வைத்த உள்ளத் தமிழர் நூற்கள்


In [17]:
wer_metric = load_metric("wer")
without_lm_wer = wer_metric.compute(predictions=data_frame["without_lm"], references=data_frame["text"])
with_lm_wer = wer_metric.compute(predictions=data_frame["with_lm"], references=data_frame["text"])

In [18]:
without_lm_wer 

0.6505550091330616

In [19]:
with_lm_wer

0.20331600393424196